# Transportation Emissions Exposure Analysis
Sum of daily exposure to emissions from transportation sources using ABM outputs

In [67]:
%matplotlib inline
import pandas as pd
import numpy as np

In [81]:
trip = pd.read_csv('_trip.tsv', sep='\t')


## Use trip records to create activity patterns for each simulated person in the region

In [2]:
df = pd.read_csv('_trip.tsv', sep='\t')

# Generate unique person ID field 
df['person_id'] = df['hhno'].astype('str') + '_' + df['pno'].astype('str')

# Convert individual trips to daily activity
# e.g., work -> home at 7 am and home to work at 5 pm becomes:
# home from 12 am to 7 am, travel between 7 and 7:30, at work location from 7:30 to 5pm, travel from 5 to 5:30, home until 12am

# Build an activity dataframe for each person
activity = pd.DataFrame()
activity.index = [[],[]]

# Loop through each simulated person-day
for person_id in df['person_id'].unique()[0:10]:    # Using sample for testing
    trip_subsample = df[df['person_id'] == person_id]    # Isolate list of trips for a person
    for row in xrange(0, len(trip_subsample)-1):    # Loop trhough each trip to identify activity
        row_index = 0    
        current_row = trip_subsample.iloc[row]    # For comparison purposes, define current row vs next
        next_row = trip_subsample.iloc[row+1]

        # Activity from midnight until first trip
        if row == 0:    # First trip indicates where person was before they left (often at home)
            activity.ix[(person_id,0),'person_id'] = person_id
            activity.ix[(person_id,0),'activity'] = current_row['opurp']
            activity.ix[(person_id,0),'activity_index'] = 0
            activity.ix[(person_id,0),'parcel'] = current_row['opcl']
            activity.ix[(person_id,0),'begin_time'] = 0 # minutes after midnight
            activity.ix[(person_id,0),'end_time'] = current_row['deptm']
            activity.ix[(person_id,0),'duration'] = activity.ix[(person_id,0),'end_time']-activity.ix[(person_id,0),'begin_time']
            row_index += 1
       
        # Second activity
        activity.ix[(person_id,row+row_index),'person_id'] = person_id
        activity.ix[(person_id,row+row_index),'activity'] = current_row['dpurp']
        activity.ix[(person_id,row+row_index),'activity_index'] = row+row_index
        activity.ix[(person_id,row+row_index),'parcel'] = next_row['opcl']
        activity.ix[(person_id,row+row_index),'begin_time'] = current_row['arrtm']
        activity.ix[(person_id,row+row_index),'end_time'] = next_row['deptm'] # minutes after midnight
        activity.ix[(person_id,row+row_index),'duration'] = activity.ix[(person_id,row+row_index),'end_time']-activity.ix[(person_id,row+row_index),'begin_time']
            
        # for last trip of the day
        if row == len(trip_subsample)-2:
            activity.ix[(person_id,row+2),'activity_index'] = row+2
            activity.ix[(person_id,row+2),'activity'] = next_row['dpurp']
            activity.ix[(person_id,row+2),'person_id'] = person_id
            activity.ix[(person_id,row+2),'parcel'] = next_row['dpcl']
            activity.ix[(person_id,row+2),'begin_time'] = next_row['arrtm'] # minutes after midnight
            activity.ix[(person_id,row+2),'end_time'] = 1440 # minutes after midnight
            activity.ix[(person_id,row+2),'duration'] = 1440-activity.ix[(person_id,row+2),'begin_time']

In [3]:
##### Above algorithm works, but is extremely slow. 
# Needs to either be threaded or vectorized

In [4]:
# further separate each activity to 12 time periods to get hourly air quality estimates
# use floor to define the bin
activity['begin_hour'] = np.floor(activity['begin_time']/60.0).astype('int')
activity['end_hour'] = np.floor(activity['end_time']/60.0).astype('int')

In [156]:
# Keep track of what percent of activity occurred during this hour
activity['begin_hour_fraction'] = (activity['begin_time']-(activity['begin_hour']*60))/60
activity['end_hour_fraction'] = (activity['end_time']-((activity['end_hour'])*60))/60

In [157]:
activity.to_csv('c:/users/brice/person_activity.csv',index=False)

In [531]:
activity = pd.read_csv(r'c:/users/brice/person_activity.csv')

In [159]:
block_parcel_lookup = pd.read_csv(r'R:\aq\new_parcel_block_lookup.txt')

In [160]:
# activity = pd.merge(activity, block_parcel_lookup[['GEOID10','parcelid']], left_on='parcel', right_on='parcelid',
#                    how='left')

In [7]:
# Calcualte total hourly emission exposure for each parcel, by time period

# Load intersection of blocks and network components (replace with parcel intersect in future)
block_network = pd.read_csv(r'R:\aq\block_network_intersect.txt')

In [250]:
# Shrink the block_network file and rename as df

df = block_network[['Shape_Length','GEOID10','NewINode','NewJNode']]
# Remove any block that doesn't exist in activity dataframe
df = df[df['GEOID10'].isin(pd.unique(activity['GEOID10']))]

In [251]:
# Note that not all blocks contain parcels - user an inner join
# df = pd.merge(block_network, block_parcel_lookup[['parcelid','GEOID10']], on='GEOID10', how='inner')

In [252]:
# Load air quality results and join with parcel information
aq_rates = pd.read_csv(r'L:\T2040\soundcast_2014\outputs\aq_2014_july.csv')
pollutant_list = [1,2,3,5,6,79,87,90,91,98,100,106,107,110,112,115,116,117,118,119]
aq_rates = aq_rates[['inode_x','jnode_x','total_volume','hourId']+[str(i) for i in pollutant_list]]

In [253]:
block_rates = pd.merge(df, aq_rates, left_on=['NewINode','NewJNode'], right_on=['inode_x','jnode_x'], how='left')

In [254]:
for pollutant in pollutant_list:
    block_rates[str(pollutant)+'_total_grams'] = block_rates['total_volume']*(block_rates['Shape_Length']/5280)*block_rates[str(pollutant)]


In [566]:
# Sum of emissions emitted into each block by hour
total_hourly_block_grams = block_rates.groupby(['GEOID10','hourId']).sum()[['100_total_grams','1_total_grams']]
total_hourly_block_grams = total_hourly_block_grams.reset_index()

# Pollution totals are calculated by assignment period
Take the average for each hour to calcualte hourly exposure

In [581]:
def average_emissions_to_hours(df, hour_list):
    
    # First hour contains information summed for all time periods
    copy_df = df[df['hourId'] == hour_list[0]].copy()
    
    for hour in hour_list:
        _df = copy_df.copy()
#         print hour
        if hour == hour_list[0]:
            df = df[df['hourId'] != hour_list[0]]
        _df['hourId'] = hour
        _df[['100_total_grams','1_total_grams']]/(len(hour_list))
        df = df.append(_df)
        
    df = df.reset_index()
    df = df.drop('index', axis=1)
    return df

In [586]:
hourly_emissions_total = average_emissions_to_hours(total_hourly_block_grams, hour_list=[10,11,12,13])
hourly_emissions_total = average_emissions_to_hours(hourly_emissions_total, hour_list=[18,19])
hourly_emissions_total = average_emissions_to_hours(hourly_emissions_total, hour_list=[20,21,22,23,0,1,2,3,4])


In [587]:
# Write to file ?
hourly_emissions_total.to_csv(r'R:/aq/hourly_emissions_total.csv', index=False)

## Calculate hourly totals by activity
total_hourly_block_grams is the sum of emissions (grams) released during each hour by block
We need to join this to the activity list to get the total per person per day


In [259]:
# Start with a single person's activity
activity_sample  = activity[activity['person_id'] == '9_1']

In [588]:
# Calculate the total grams for a given block and time period

def total_activity_emissions(df, zone_num, emissions_type, begin_hour, begin_hour_share, end_hour, end_hour_share,
                            geography_field='GEOID10'):
    """Calculate the total grams per each activity"""
    
    df = hourly_emissions_total
    
    # Totals from first hour
    first_hour_total = df[(df[geography_field] == zone_num) & (df.hourId == begin_hour)][emissions_type].values[0]
    first_hour_total = first_hour_total*begin_hour_share    # Modify with % of hour at that location
    
    # Totals from last hour
    last_hour_total = df[(df[geography_field] == zone_num) & (df.hourId == end_hour)][emissions_type].values[0]
    last_hour_total = last_hour_total*end_hour_share    # Modify with % of hour at that location
    
    # Calculate totals for interim hours if necessary
    interim_total = 0
    if end_hour-begin_hour>1:
        for hour in xrange(begin_hour+1,end_hour):
            interim_total +=  df[(df[geography_field] == zone_num) & (df.hourId == hour)][emissions_type].values[0]
            
    activity_total = first_hour_total + interim_total + last_hour_total
    
    return activity_total

,person_id,activity,activity_index,parcel,begin_time,end_time,duration,begin_hour,end_hour,GEOID10,parcelid,begin_hour_fraction,end_hour_fraction,total_exposure
10,14_1,4.0,1.0,1075189.0,888.0,896.0,8.0,14,14,530330050002007,1075189,0.800000,0.933333,NaN
13,1_1,3.0,1.0,1201491.0,811.0,869.0,58.0,13,14,530330032002011,1201491,0.516667,0.483333,NaN
16,3_1,7.0,1.0,991506.0,1179.0,1373.0,194.0,19,22,530330002002015,991506,0.650000,0.883333,NaN
19,7_1,4.0,1.0,1070691.0,762.0,872.0,110.0,12,14,530330205002003,1070691,0.700000,0.533333,NaN
23,11_1,5.0,2.0,1161036.0,812.0,995.0,183.0,13,16,530330001004002,1161036,0.533333,0.583333,NaN
26,23_1,4.0,1.0,1022273.0,514.0,557.0,43.0,8,9,530330217005016,1022273,0.566667,0.283333,NaN
30,9_1,5.0,2.0,913611.0,664.0,785.0,121.0,11,13,530330012004011,913611,0.066667,0.083333,NaN


In [614]:
# I can't figure out how to use lambda functino for a full dataframe right now,
# so let's loop for now ...

# Only include activities that occur within areas that have pollution
df = activity[activity['GEOID10'].isin(pd.unique(hourly_emissions_total['GEOID10']))]

results = []
for i in xrange(len(df)):
    row = df.iloc[i]
    tot_emissions = total_activity_emissions(df, zone_num=row['GEOID10'], emissions_type='1_total_grams', 
                             begin_hour=row['begin_hour'], begin_hour_share=row['begin_hour_fraction'], 
                             end_hour=row['end_hour'], end_hour_share=row['end_hour_fraction'])
    results.append(tot_emissions)
    
df['total_exposure'] = results

In [617]:
df.groupby('person_id').sum()

,activity,activity_index,parcel,begin_time,end_time,duration,begin_hour,end_hour,GEOID10,parcelid,begin_hour_fraction,end_hour_fraction,total_exposure
person_id,,,,,,,,,,,,,
11_1,5.0,2.0,1161036.0,812.0,995.0,183.0,13,16,530330001004002,1161036,0.533333,0.583333,6.712978
14_1,4.0,1.0,1075189.0,888.0,896.0,8.0,14,14,530330050002007,1075189,0.800000,0.933333,64.069877
1_1,3.0,1.0,1201491.0,811.0,869.0,58.0,13,14,530330032002011,1201491,0.516667,0.483333,22.137184
23_1,4.0,1.0,1022273.0,514.0,557.0,43.0,8,9,530330217005016,1022273,0.566667,0.283333,34.372314
3_1,7.0,1.0,991506.0,1179.0,1373.0,194.0,19,22,530330002002015,991506,0.650000,0.883333,24.299103
7_1,4.0,1.0,1070691.0,762.0,872.0,110.0,12,14,530330205002003,1070691,0.700000,0.533333,14.562449
9_1,5.0,2.0,913611.0,664.0,785.0,121.0,11,13,530330012004011,913611,0.066667,0.083333,142.562655


In [619]:
# WOOHOO - total daily emissions exposure by person!

# For Analysis:
- attach activity type to each location
- other things

In [618]:
df

,person_id,activity,activity_index,parcel,begin_time,end_time,duration,begin_hour,end_hour,GEOID10,parcelid,begin_hour_fraction,end_hour_fraction,total_exposure
10,14_1,4.0,1.0,1075189.0,888.0,896.0,8.0,14,14,530330050002007,1075189,0.800000,0.933333,64.069877
13,1_1,3.0,1.0,1201491.0,811.0,869.0,58.0,13,14,530330032002011,1201491,0.516667,0.483333,22.137184
16,3_1,7.0,1.0,991506.0,1179.0,1373.0,194.0,19,22,530330002002015,991506,0.650000,0.883333,24.299103
19,7_1,4.0,1.0,1070691.0,762.0,872.0,110.0,12,14,530330205002003,1070691,0.700000,0.533333,14.562449
23,11_1,5.0,2.0,1161036.0,812.0,995.0,183.0,13,16,530330001004002,1161036,0.533333,0.583333,6.712978
26,23_1,4.0,1.0,1022273.0,514.0,557.0,43.0,8,9,530330217005016,1022273,0.566667,0.283333,34.372314
30,9_1,5.0,2.0,913611.0,664.0,785.0,121.0,11,13,530330012004011,913611,0.066667,0.083333,142.562655


In [109]:
# MATRIX APPROACH may be useful?
# # Split activities into equal 24 hour portions
# # Create a table for each person and each parcel location, at 24 hour intervals
# # A Persons's daily exposure will be the sum of their tables

# # A dataframe of percent of hour spent in parcel location for each person

# person_results = {}

# person_list = pd.unique(activity['person_id']) # temporarily reduce list zize
# for person_id in person_list:
#     person_results[person_id] = {}
    
#     activity_list = activity[activity['person_id'] == person_id]
    
#     parcel_hourly_activity = pd.DataFrame()
#     parcel_hourly_activity.index = [[],[]]
#     parcel_hourly_activity['time_in_parcel'] = 0
    
#     # Produce matrix for each parcel
#     for parcel in pd.unique(activity_list.parcel):
#         parcel_activity_list = activity_list[activity_list['parcel'] == parcel]
        
#         for row in xrange(len(parcel_activity_list)):
#             activity_row = parcel_activity_list.iloc[row]
#             for hour in xrange(activity_row.begin_hour,activity_row.end_hour+1):
#                 parcel_hourly_activity['parcel'] = parcel
#                 parcel_hourly_activity.ix[(activity_row['person_id'],hour),'hour'] = hour
#                 parcel_hourly_activity.ix[(activity_row['person_id'],hour),'parcel'] = activity_row.parcel
#                 parcel_hourly_activity.ix[(activity_row['person_id'],hour),'person_id'] = activity_row.person_id
#                 if activity_row['end_time']/60 > hour:
#                     end_share = (activity_row['end_time'] - hour*60.0)/60
#                 else:
#                     end_share = 1
#                 if activity_row['begin_time']/60 > hour:
#                     begin_share = (activity_row['begin_time'] - hour*60.0)/60
#                 else:
#                     begin_share = 0
#                 final_share = end_share-begin_share
                
#                 parcel_hourly_activity.ix[(activity_row['person_id'],hour),'time_in_parcel'] = final_share
        
#         person_results[person_id][parcel] = parcel_hourly_activity
        
    
#     for row in xrange(len(activity_list)):
#         activity_row = activity_list.iloc[row]
# #         print '---------'
#         for hour in xrange(activity_row.begin_hour,activity_row.end_hour+1):
# #             print hour
#             hourly_activity.ix[(activity_row['person_id'],hour),'hour'] = hour
#             hourly_activity.ix[(activity_row['person_id'],hour),'parcel'] = activity_row.parcel
#             hourly_activity.ix[(activity_row['person_id'],hour),'person_id'] = activity_row.person_id
            
#             if hour == activity_row.begin_hour:
                
            
#             if (hour == activity_row.begin_hour) or (hour == activity_row.end_hour):
#                 hour_fraction = max(activity_row.begin_hour_fraction, activity_row.end_hour_fraction)
#                 hourly_activity.ix[(activity_row['person_id'],hour),'hour_fraction'] = hour_fraction
#             else:
#                 hourly_activity.ix[(activity_row['person_id'],hour),'hour_fraction'] = 1

In [32]:
# Reshape volumes to be for all 24 hours
# dictionary to relate time period to 24 hours
tod_dict = {0: '20to5',
    1: '20to5',
    2: '20to5',
    3: '20to5',
    4: '20to5',
    5: '5to6',
    6: '6to7',
    7: '7to8',
    8: '8to9',
    9: '9to10',
    10: '10to14',
    11: '10to14',
    12: '10to14',
    13: '10to14',
    14: '14to15',
    15: '15to16',
    16: '16to17',
    17: '17to18',
    18: '18to20',
    19: '18to20',
    20: '20to5',
    21: '20to5',
    22: '20to5',
    23: '20to5'
}

In [33]:
# Convert from single hour to time of day period for Soundcast
hourly_activity['tod'] = hourly_activity['hour'].astype('int')
hourly_activity.replace({"tod":tod_dict},inplace=True)

In [34]:
hourly_activity['parcel'] = hourly_activity['parcel'].astype('int')

In [35]:
# Drop parcels that don't show up in hourly_activity file to reduce file size
parcel_vol_trimmed = parcel_vol[parcel_vol['parcelid'].isin(pd.unique(hourly_activity.parcel))]

In [39]:
# Join the link volumes to the activity
_df = pd.merge(hourly_activity,parcel_vol_trimmed,left_on=['parcel','tod'],right_on=['parcelid','tod'])
_df['hour'] = _df.hour.astype('int')

In [54]:
_df[_df['person_id'] == '23_1'].to_csv('C:/users/brice/test.csv')

In [49]:
hourly_activity[hourly_activity['person_id'] == '23_1']

hour   parcel person_id     tod
23_1 0.0    0.0  1144246      23_1   20to5
     1.0    1.0  1144246      23_1   20to5
     2.0    2.0  1144246      23_1   20to5
     3.0    3.0  1144246      23_1   20to5
     4.0    4.0  1144246      23_1   20to5
     5.0    5.0  1144246      23_1    5to6
     6.0    6.0  1144246      23_1    6to7
     7.0    7.0  1144246      23_1    7to8
     8.0    8.0  1022273      23_1    8to9
     9.0    9.0  1144246      23_1   9to10
     10.0  10.0  1144246      23_1  10to14
     11.0  11.0  1144246      23_1  10to14
     12.0  12.0  1144246      23_1  10to14
     13.0  13.0  1144246      23_1  10to14
     14.0  14.0  1144246      23_1  14to15
     15.0  15.0  1144246      23_1  15to16
     16.0  16.0  1144246      23_1  16to17
     17.0  17.0  1144246      23_1  17to18
     18.0  18.0  1144246      23_1  18to20
     19.0  19.0  1144246      23_1  18to20
     20.0  20.0  1144246      23_1   20to5
     21.0  21.0  1144246      23_1   20to5
     22.0  22.0  1144246      23_1   20to5
     23.0  23.0  1144246      23_1   20to5
     24.0  24.0  1144246      23_1      24

In [55]:
# Add in the hourly rates to compute total emissions accumulated over time

# Load some test rates
rates = pd.read_csv(r'L:\T2040\soundcast_2014\outputs\aq_2014_july.csv')

# Need to use real MOVES rates here: 
# e.g., L:\T2040\soundcast_2014\scripts\summarize\inputs\network_summary\emission_rates_2014.csv

In [56]:
rates.head()

,Unnamed: 0,inode_x,jnode_x,ij,length,posted_speed,congested_speed,facility_type,total_volume,sov_vol,...,100,106,107,110,112,115,116,117,118,119
0,753111,1,59289,1-59289,0.310,15.0,15.0,5.0,32.820129,23.214807,...,0.058596,0.104287,0.015462,0.053541,0.0216,0.00159,0.013036,0.002319,0.03194,0.0
1,753112,1,59975,1-59975,0.105,15.0,15.0,5.0,64.634285,44.834290,...,0.058596,0.104287,0.015462,0.053541,0.0216,0.00159,0.013036,0.002319,0.03194,0.0
2,753113,2,59395,2-59395,0.146,15.0,15.0,5.0,8.000000,4.000000,...,0.058596,0.104287,0.015462,0.053541,0.0216,0.00159,0.013036,0.002319,0.03194,0.0
3,753114,2,59419,2-59419,0.121,15.0,15.0,5.0,219.342056,137.095276,...,0.058596,0.104287,0.015462,0.053541,0.0216,0.00159,0.013036,0.002319,0.03194,0.0
4,753115,3,58237,3-58237,0.227,15.0,15.0,5.0,100.763458,66.820068,...,0.058596,0.104287,0.015462,0.053541,0.0216,0.00159,0.013036,0.002319,0.03194,0.0


In [57]:
newdf = pd.merge(_df,rates,left_on=['hour','ij'], right_on=['hourId','ij'], how='left')
newdf = pd.merge(_df,rates,left_on=['hour','ij'], right_on=['hourId','ij'])

In [58]:
# Not sure what the columns mean for now; just test
newdf['pm_10'] = newdf['total_volume']*newdf['100']

In [59]:
newdf.groupby('person_id').sum()['pm_10']

person_id
3_1    1897.47779
Name: pm_10, dtype: float64

In [ ]:
# By attaching this to the person record we can analyze results by household, workplace location,
# worker type, income, (equity geographies)

In [ ]:
# Should also add the impacts incurred while driving ???
# Will be harder because changes are instantaneous
# On a link for 5 seconds while driving, involves much more fine-grained calculations
# The initial numbers can be stationary emissions exposure, considered a baseline


In [61]:
newdf.to_csv('C:/users/brice/test.csv')

In [ ]:
# Trim activity dataframe to only include hours 5-20
# # Ideally we calculate for all hours of the day
# # We are also missing 10-2 PM
# activity_trimmed = activity.copy()

# # Entirely drop activities that start and end outside of 0-5 am and from 20 to 24
# activity_trimmed = activity_trimmed[-((activity_trimmed['begin_hour'] < 5) & (activity_trimmed['end_hour'] < 5))]
# activity_trimmed = activity_trimmed[-((activity_trimmed['begin_hour'] > 20) & (activity_trimmed['end_hour'] > 20))]

# # For the remaining activities, at least one part of the activity is within the 5-20 time period 
# # For these activities set actiovity start at 5 am and end at 20, and reset fractions of hours accordingly
# activity_trimmed.ix[activity_trimmed['begin_hour'] < 5, 'begin_time'] = 5*60
# activity_trimmed.ix[activity_trimmed['begin_hour'] < 5, 'begin_hour_fraction'] = 1
# activity_trimmed.ix[activity_trimmed['begin_hour'] < 5, 'begin_hour'] = 5

# activity_trimmed.ix[activity_trimmed['end_hour'] > 20, 'end_time'] = 20*60
# activity_trimmed.ix[activity_trimmed['end_hour'] > 20, 'end_hour_fraction'] = 1
# activity_trimmed.ix[activity_trimmed['end_hour'] > 20, 'end_hour'] = 20


# # End activity at 10
# activity_trimmed.ix[(activity_trimmed['begin_hour'] < 10) & (activity_trimmed['end_hour'] < 14), 'end_time'] = 10*60
# activity_trimmed.ix[(activity_trimmed['begin_hour'] < 10) & (activity_trimmed['end_hour'] < 14), 'end_hour_fraction'] = 1
# activity_trimmed.ix[(activity_trimmed['begin_hour'] < 10) & (activity_trimmed['end_hour'] < 14), 'end_hour'] = 10

# # Begin activity at 14
# activity_trimmed.ix[(activity_trimmed['end_hour'] > 14) & (activity_trimmed['begin_hour'] > 10), 'begin_time'] = 14*60
# activity_trimmed.ix[(activity_trimmed['end_hour'] > 14) & (activity_trimmed['begin_hour'] > 10), 'begin_hour_fraction'] = 1
# activity_trimmed.ix[(activity_trimmed['end_hour'] > 14) & (activity_trimmed['begin_hour'] > 10), 'begin_hour'] = 14

# # Trip that start before 10 and end after 14
# ############## INCORRECT RIGHT NOW, NEED FULL DAY RATES
# # TRUNCATE AT 10!!!
# activity_trimmed.ix[(activity_trimmed['begin_hour'] < 10) & (activity_trimmed['end_hour'] > 14), 'end_time'] = 10*60
# activity_trimmed.ix[(activity_trimmed['begin_hour'] < 10) & (activity_trimmed['end_hour'] > 14), 'end_hour_fraction'] = 1
# activity_trimmed.ix[(activity_trimmed['begin_hour'] < 10) & (activity_trimmed['end_hour'] > 14), 'end_hour'] = 10


# # Activities starting  between 10 and 14
# activity_trimmed = activity_trimmed[-((activity_trimmed['begin_hour'] > 10) & (activity_trimmed['end_hour'] <= 14))]

# # Filter to remove any activities that occur with blocks that have no emissions
# # These appear during the intersection to calculate hourly totals
# # If a block is not near a roadway, there will be zero emissions

# # For activities in areas that don't show up in the total_hourly_block grams dataframe, set total exposure to 0
# activity_trimmed.ix[-activity_trimmed['GEOID10'].isin(pd.unique(total_hourly_block_grams['GEOID10'])), 'total_exposure'] = 0

# Sum of Exposure While Traveling